In [1]:
import torchvision
import torch
import timm

/home/shawnman99/anaconda3/envs/shawn99/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#device = 'cpu'
device = 'cuda:3'

In [3]:
PATH = 'data/resnet18_total(1+2)_best.pt'
trained = timm.create_model('resnet18', pretrained=False, num_classes = 6)
trained.load_state_dict(torch.load(PATH, map_location=device))
trained.to(device)
print(trained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [4]:
from typing import List, Callable, Tuple

import cv2
import torch
from torch.utils.data import Dataset

class BaseSet(Dataset):
    def __init__(self,
                 paths: List[str],
                 labels: List[int],
                 transforms: Callable = None,
                 ) -> None:
        super().__init__()
        self.paths = paths
        self.labels = labels
        self.transforms = transforms
        
    def __getitem__(self,
                    index,
                    ) -> Tuple[torch.Tensor, torch.Tensor]:
        path = self.paths[index]
        label = self.labels[index]
        
        img = cv2.imread(str(path))
        if self.transforms:
            img = self.transforms(img)
        
        label = torch.tensor(label)
        img = img.to(device)
        label = label.to(device)
        return img, label    
    def __len__(self):
        return len(self.paths)

In [5]:
from pathlib import Path
ROOT = Path('/home/shawnman99/calibration')
DATA_DIR = ROOT / "data"
CLASS_NAMES = ['crack', 'ddul', 'imul', 'ok', 'scratch', 'void']
test_paths = [p for p in (DATA_DIR / f'total(1+2)_split/valid').glob('*/*')
                if p.suffix in ['.png', '.jpg'] and p.parent.name in CLASS_NAMES]
test_labels = [CLASS_NAMES.index(p.parent.name) for p in test_paths]

In [6]:
import torchvision.transforms as trs
from torch.utils.data import DataLoader
testset = BaseSet(test_paths, test_labels, trs.Compose([
    trs.ToTensor(),
    trs.Resize((384,384))
]))
testloader = DataLoader(testset, batch_size=4)

In [7]:
import calibration_library.metrics as metrics
from torch.nn import functional as F
from scipy.special import softmax
correct = 0
total = 0

# First: collect all the logits and labels for the validation set
logits_list = []
labels_list = []

trained.eval()
with torch.no_grad():
    for images, labels in testloader:
        #outputs are the the raw scores!
        logits = trained(images)
        #add data to list
        logits_list.append(logits)
        labels_list.append(labels)
        #convert to probabilities
        output_probs = F.softmax(logits,dim=1)
        #get predictions from class
        probs, predicted = torch.max(output_probs, 1)
        #total
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    logits = torch.cat(logits_list)
    labels = torch.cat(labels_list)

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
print(total)

################
#metrics

ece_criterion = metrics.ECELoss()
#Torch version
logits_np = logits.cpu().numpy()
labels_np = labels.cpu().numpy()
"""
#Numpy Version
print('ECE: %f' % (ece_criterion.loss(logits_np,labels_np, 15)))

softmaxes = softmax(logits_np, axis=1)

print('ECE with probabilties %f' % (ece_criterion.loss(softmaxes,labels_np,15,False)))

mce_criterion = metrics.MCELoss()
print('MCE: %f' % (mce_criterion.loss(logits_np,labels_np)))

oe_criterion = metrics.OELoss()
print('OE: %f' % (oe_criterion.loss(logits_np,labels_np)))

sce_criterion = metrics.SCELoss()
print('SCE: %f' % (sce_criterion.loss(logits_np,labels_np, 15)))

ace_criterion = metrics.ACELoss()
print('ACE: %f' % (ace_criterion.loss(logits_np,labels_np,15)))

tace_criterion = metrics.TACELoss()
threshold = 0.01
print('TACE (threshold = %f): %f' % (threshold, tace_criterion.loss(logits_np,labels_np,threshold,15)))
"""

Accuracy of the network on the 10000 test images: 99 %
31075


"\n#Numpy Version\nprint('ECE: %f' % (ece_criterion.loss(logits_np,labels_np, 15)))\n\nsoftmaxes = softmax(logits_np, axis=1)\n\nprint('ECE with probabilties %f' % (ece_criterion.loss(softmaxes,labels_np,15,False)))\n\nmce_criterion = metrics.MCELoss()\nprint('MCE: %f' % (mce_criterion.loss(logits_np,labels_np)))\n\noe_criterion = metrics.OELoss()\nprint('OE: %f' % (oe_criterion.loss(logits_np,labels_np)))\n\nsce_criterion = metrics.SCELoss()\nprint('SCE: %f' % (sce_criterion.loss(logits_np,labels_np, 15)))\n\nace_criterion = metrics.ACELoss()\nprint('ACE: %f' % (ace_criterion.loss(logits_np,labels_np,15)))\n\ntace_criterion = metrics.TACELoss()\nthreshold = 0.01\nprint('TACE (threshold = %f): %f' % (threshold, tace_criterion.loss(logits_np,labels_np,threshold,15)))\n"

In [21]:
with open('logits.txt', 'w') as f:
    for l in logits_np:
        ll = [str(x) for x in l]
        f.write(",".join(ll)+'\n')

In [22]:
with open('labels.txt', 'w') as f:
    for l in labels:
        f.write(",".join(str(int(l))) + "\n")

In [8]:
from calibration_library.recalibration import Recalibration
Histo = Recalibration(logits_np, labels_np)
Histo.fit_HistBin()
print(Histo.histBin)
print(Histo.n_histBin)

[[0.00013116044, 0.125, 0.0, 0.0, 0.4, 0, 0.0, 0.33333334, 0.4, 0.0, 0.6666667, 0.5, 0.6666667, 0.71428573, 0.99040306], [0.0013049294, 0.11111111, 0.16666667, 0.5, 0.4, 0.5, 0.42857143, 0.33333334, 0.75, 0.6, 0.25, 0.33333334, 0.6, 0.9166667, 0.9690403], [0.0010726126, 0.26086956, 0.5, 0.5, 0.5, 0.33333334, 0.5, 0.5714286, 0.33333334, 0.5, 0.5, 1.0, 0.75, 0.7, 0.92990655], [0.0027995522, 0.0, 0.2, 0.0, 1.0, 0.25, 0.36363637, 0.0, 0.2, 1.0, 0.5555556, 0.75, 0.33333334, 0.5555556, 0.998802], [0.00035813122, 0.3, 0.375, 0.33333334, 0.6, 0.0, 1.0, 1.0, 0.6666667, 1.0, 0.0, 0.75, 1.0, 1.0, 0.99041533], [0.0005534935, 0.1764706, 0.33333334, 0.375, 0.4, 0.0, 0.6, 0.25, 0.4, 0.6666667, 0.4, 0.6666667, 0.5, 0.8333333, 0.90909094]]
15


In [11]:
import calibration_library.visualization as visualization
cal_p = [Histo.calibrate_HistBin(x) for x in logits_np]
rel_diagram = visualization.ReliabilityDiagram()
plt_test_2 = rel_diagram.pplot(cal_p, labels_np,title="Reliability Diagram")
#plt_test_2.savefig('plots/rel_diagram_test.png',bbox_inches='tight')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
print(cal_p)

[array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([6.6666669e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 3.7500000e-01], dtype=float32), array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([9.9040306e-01, 1.3049294e-03, 1.0726126e-03, 2.7995522e-03,
       3.5813122e-04, 5.5349350e-04], dtype=float32), array([9.9040306e-01, 1.3049294